# La agencia de viajes

Supongamos que somos un agente de Airbnb localizado en CDMX, y tenemos que atender peticiones de varios clientes. Tenemos un archivo llamado airbnb_cdmx.csv donde tenemos información de todos los alojamientos de Airbnb en CDMX

En concreto el dataset tiene las siguientes variables:

* id: el identificador de la propiedad
* host_id: el identificador del dueño de la propiedad
* neighborhood_cleansed: alcaldía en CDMX
* property_type: tipo de propiedad 
* room_type: tipo de habitación
* accommodates: El numero de personas que se pueden alojar en la propiedad
* price: precio (en pesos mexicanos)
* minimum_nights, maximum_nights: mínimo y máximo de noches
* bedrooms: El número de habitaciones
* beds: número de camas
* cancellation_policy: política de cancelación
* number_of_reviews: El numero de opiniones
* review_scores_accuracy: Puntuacion media del apartamento

In [1]:
import numpy as np
import pandas as pd

In [2]:
airbnb = pd.read_csv("airbnb_cdmx.csv")
airbnb.head()

,id,host_id,neighbourhood_cleansed,property_type,room_type,accommodates,price,minimum_nights,maximum_nights,bedrooms,beds,cancellation_policy,number_of_reviews,review_scores_accuracy
0,35797,153786,Cuajimalpa de Morelos,Villa,Entire home/apt,2,4500,1,7,1.0,1.0,flexible,0,NaN
1,56074,265650,Cuauhtemoc,Condominium,Entire home/apt,3,843,4,150,1.0,2.0,moderate,60,10.0
2,61792,299558,Cuauhtemoc,House,Private room,2,1359,2,21,1.0,1.0,moderate,52,10.0
3,70644,212109,Coyoacan,Apartment,Entire home/apt,2,1245,6,180,1.0,1.0,moderate,102,10.0
4,107078,540705,Miguel Hidalgo,Loft,Entire home/apt,2,4868,4,365,1.0,1.0,strict_14_with_grace_period,10,10.0


## Alicia y su familia

Alicia va a ir a CDMX durante una semana con su marido y sus 2 hijos. Están buscando un apartamento con habitaciones separadas para los padres y los hijos. No les importa donde alojarse o el precio, simplemente quieren tener una experiencia agradable. Esto significa que solo aceptan lugares con más de 10 críticas con una puntuación mayor de 4. Cuando seleccionemos habitaciones para Alicia, tenemos que asegurarnos de ordenar las habitaciones de mejor a peor puntuación. Para aquellas habitaciones que tienen la misma puntuación, debemos mostrar antes aquellas con más críticas. Debemos a lo más la tercera parte del total de posibilidades.

In [3]:
""" 
Filtrar:
 noches_estancia = 6
 habitaciones >= 2
 num_criticas > 10
 puntuacion > 4
Ordenar descendente por puntuación
Misma puntuación, ordenar por num_críticas descendente
Mostrar solo 1/3 de los resultados
"""

# Noches:
# Se quiere quedar exactamente 6 noches.
# Las propiedades que le sirven son aquellas que:
# - no pidan un mínimo mayor a 6 y
# - no tengan un máximo menor a 6.
f1 = airbnb["maximum_nights"] >= 6
f2 = airbnb["minimum_nights"] <= 6
# Habitaciones separadas para papás e hijos
f3 = airbnb["bedrooms"] >= 2
# Mínimo 4 personas
f4 = airbnb["accommodates"] >= 4
# Mínimo 10 críticas
f5 = airbnb["number_of_reviews"] > 10
# Puntuación mínima de 4
f6 = airbnb["review_scores_accuracy"] > 4
# Combinar filtros
opciones = airbnb[f1 & f2 & f3 & f4 & f5 & f6]
opciones

,id,host_id,neighbourhood_cleansed,property_type,room_type,accommodates,price,minimum_nights,maximum_nights,bedrooms,beds,cancellation_policy,number_of_reviews,review_scores_accuracy
6,165772,790208,Miguel Hidalgo,House,Entire home/apt,14,1245,2,365,4.0,10.0,strict_14_with_grace_period,266,10.0
9,187030,899360,Cuauhtemoc,Apartment,Entire home/apt,6,1314,3,365,2.0,3.0,moderate,97,10.0
12,198778,970609,Cuauhtemoc,Apartment,Entire home/apt,4,1087,2,365,2.0,4.0,strict_14_with_grace_period,169,9.0
15,246057,1288063,Cuauhtemoc,Apartment,Entire home/apt,5,1699,1,30,2.0,4.0,flexible,247,10.0
19,256555,790208,Miguel Hidalgo,House,Entire home/apt,14,1245,2,365,4.0,10.0,strict_14_with_grace_period,264,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19714,42202314,132484396,Benito Juarez,Apartment,Entire home/apt,5,725,2,1125,2.0,3.0,strict_14_with_grace_period,12,10.0
19741,42224748,335393886,Miguel Hidalgo,Condominium,Entire home/apt,4,2332,1,30,2.0,2.0,moderate,21,10.0
19771,42249246,332120390,Benito Juarez,Apartment,Entire home/apt,5,1200,1,1125,2.0,3.0,flexible,13,10.0
19800,42272228,335393886,Miguel Hidalgo,Condominium,Entire home/apt,4,2255,1,30,2.0,2.0,moderate,15,10.0


In [4]:
# Ordenar
opciones = opciones.sort_values(by=["review_scores_accuracy", "number_of_reviews"],
                     ascending=[False, False])

In [5]:
# Limitar solo a la tercera parte
num_opciones = opciones.shape[0] // 3
opciones = opciones.head(num_opciones)
opciones

,id,host_id,neighbourhood_cleansed,property_type,room_type,accommodates,price,minimum_nights,maximum_nights,bedrooms,beds,cancellation_policy,number_of_reviews,review_scores_accuracy
2800,14150131,10644799,Venustiano Carranza,Loft,Entire home/apt,4,473,1,1125,2.0,3.0,flexible,506,10.0
6024,21052296,151657679,Iztacalco,Apartment,Entire home/apt,4,540,1,1125,2.0,2.0,flexible,424,10.0
581,4801896,20440301,Cuauhtemoc,Apartment,Entire home/apt,4,1993,1,150,2.0,2.0,flexible,365,10.0
2520,13398036,70937648,Cuauhtemoc,Apartment,Entire home/apt,4,1200,1,1125,2.0,2.0,moderate,342,10.0
137,997247,5479592,Cuauhtemoc,Apartment,Entire home/apt,4,1812,2,1000,2.0,2.0,strict_14_with_grace_period,333,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8739,25973891,100844511,Coyoacan,Apartment,Entire home/apt,4,900,2,180,2.0,3.0,flexible,52,10.0
9875,28409727,39346393,Miguel Hidalgo,Apartment,Entire home/apt,6,1449,1,1125,2.0,3.0,strict_14_with_grace_period,52,10.0
10751,30009282,202890617,Cuauhtemoc,Apartment,Entire home/apt,5,1993,2,30,3.0,3.0,moderate,52,10.0
11278,30979411,82121841,Cuauhtemoc,Apartment,Entire home/apt,7,1948,2,1125,2.0,4.0,moderate,52,10.0


## Roberto contra su hermana

Roberto es un casero que tiene una casa en Airbnb. De vez en cuando nos llama preguntando sobre cuales son las críticas de su alojamiento. Hoy está particularmente molesto, ya que su hermana Valentina ha puesto una casa en Airbnb y Roberto quiere asegurarse de que su casa tiene más críticas que las de Valentina. Tenemos que crear un dataframe con las propiedades de ambos. Las id de las casas de Roberto y Valentina son 43856289 y 107078 respectivamente. 

In [6]:
# Crear filtros
casa_roberto = airbnb["id"] == 43856289
casa_valentina = airbnb["id"] == 107078
# Aplicar filtros
airbnb[casa_roberto | casa_valentina]

,id,host_id,neighbourhood_cleansed,property_type,room_type,accommodates,price,minimum_nights,maximum_nights,bedrooms,beds,cancellation_policy,number_of_reviews,review_scores_accuracy
4,107078,540705,Miguel Hidalgo,Loft,Entire home/apt,2,4868,4,365,1.0,1.0,strict_14_with_grace_period,10,10.0
21819,43856289,63979434,Cuauhtemoc,House,Private room,4,996,1,15,1.0,3.0,flexible,0,NaN


## Diana "la mochilera"

Diana va a CDMX a pasar 3 noches y quiere conocer a gente nueva. Tiene un presupuesto de 1,250 pesos mexicanos para su alojamiento. Debemos buscarle las 10 propiedades más baratas, dandole preferencia a aquellas que sean habitaciones compartidas, y para aquellas viviendas compartidas debemos elegir aquellas con mejor puntuación.

In [7]:
# Tipos de habitaciones
airbnb["room_type"].unique()

array(['Entire home/apt', 'Private room', 'Hotel room', 'Shared room'],
      dtype=object)

In [3]:
# Datos
presupuesto = 1250
noches = 3

# Filtros
# Presupuesto
fp = airbnb["price"] <= presupuesto/noches
# Noches
fn = (airbnb["minimum_nights"] <= 3) & (airbnb["maximum_nights"] >= 3)
# Cuartos compartidos
fc = airbnb["room_type"] == "Shared room"

In [9]:
# Aplicar filtros
diana = airbnb[fp & fn & fc]

# Ordenar
# 1. Precio
# 2. Que sea habitación compartida
# 3. Mejor puntuación
diana = diana.sort_values(by=["price", "review_scores_accuracy"], ascending=[True, False])
# Sólo las mejores 10
diana = diana.head(10)
diana

,id,host_id,neighbourhood_cleansed,property_type,room_type,accommodates,price,minimum_nights,maximum_nights,bedrooms,beds,cancellation_policy,number_of_reviews,review_scores_accuracy
7728,23877762,179398369,Tlalpan,Loft,Shared room,1,159,1,1125,1.0,1.0,flexible,0,NaN
19417,41917043,321717003,Iztapalapa,Condominium,Shared room,3,179,1,365,1.0,0.0,flexible,0,NaN
12068,32197684,229856606,Tlalpan,House,Shared room,1,180,3,1125,1.0,1.0,moderate,1,10.0
11666,31588864,124718595,Tlahuac,Apartment,Shared room,1,180,1,1125,1.0,1.0,flexible,3,9.0
9676,28008247,211537057,Tlalpan,Apartment,Shared room,1,180,2,31,1.0,1.0,moderate,0,NaN
6345,21596500,2269083,Miguel Hidalgo,Apartment,Shared room,1,181,3,1125,1.0,1.0,flexible,2,10.0
7377,23255186,94336357,Benito Juarez,Apartment,Shared room,1,181,2,20,1.0,2.0,flexible,3,10.0
12186,32381892,238972976,Cuauhtemoc,Hostel,Shared room,16,181,1,1125,1.0,18.0,strict_14_with_grace_period,9,10.0
16977,39228937,224496196,Cuauhtemoc,Apartment,Shared room,1,181,1,90,1.0,1.0,strict_14_with_grace_period,9,10.0
17643,39997048,305453762,Cuauhtemoc,Hostel,Shared room,8,181,1,365,1.0,8.0,moderate,8,10.0


In [10]:
""" 
Al mencionar que se quiere ordenar dándole preferencia a aquellas que sean compartidas,
se implica que también pueden ser habitaciones no compartidas, entonces, hay que desechar
uno de los filtros para permitir habitaciones no compartidas.
"""
diana = airbnb[fp & fn]

# Agregamos columna auxiliar para ordenar por habitación compartida:
diana["shared"] = diana["room_type"] == "Shared room"
# Y, ahora sí, ordenar:
diana = diana.sort_values(by=["price", "shared", "review_scores_accuracy"], ascending=[True, False, False])
diana = diana.head(10)
diana

C:\Users\JorgeZaldivar\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,id,host_id,neighbourhood_cleansed,property_type,room_type,accommodates,price,minimum_nights,maximum_nights,bedrooms,beds,cancellation_policy,number_of_reviews,review_scores_accuracy,shared
6113,21212741,48001186,Miguel Hidalgo,Apartment,Private room,2,0,1,30,1.0,1.0,strict_14_with_grace_period,29,10.0,False
6138,21266125,30026600,Miguel Hidalgo,Loft,Entire home/apt,3,0,3,400,0.0,2.0,flexible,1,10.0,False
5653,20435525,64371831,Cuauhtemoc,Apartment,Entire home/apt,7,0,1,30,3.0,4.0,strict_14_with_grace_period,105,9.0,False
5602,20277868,134853067,Coyoacan,Hostel,Private room,2,0,1,1125,1.0,1.0,moderate,1,6.0,False
4738,18570933,76376610,Miguel Hidalgo,Loft,Entire home/apt,2,0,1,1125,1.0,1.0,flexible,0,NaN,False
20122,42534951,113550322,Cuauhtemoc,Hotel,Hotel room,2,0,1,365,NaN,NaN,moderate,0,NaN,False
20123,42534965,307295793,Cuauhtemoc,Boutique hotel,Hotel room,2,0,1,365,NaN,NaN,moderate,0,NaN,False
20124,42534993,307292215,Cuauhtemoc,Hotel,Hotel room,2,0,1,365,NaN,NaN,moderate,0,NaN,False
20187,42583200,305545617,Cuauhtemoc,Hotel,Hotel room,2,0,1,365,NaN,NaN,moderate,0,NaN,False
17593,39948133,126456380,Coyoacan,House,Private room,4,23,1,1125,1.0,2.0,moderate,5,10.0,False


In [4]:
# Obtuvimos propiedades con precio 0,
# algo parece estar mal con los registros.

# Vamos a eliminar esos registros modificando el filtro de precios.
fp = (airbnb["price"] <= presupuesto/noches) & (airbnb["price"] > 0)

In [5]:
# Y repetimos todo lo anterior
# Filtrar
diana = airbnb[fp & fn].copy()  # <-- Y de una vez nos quitamos el _warning_ de arriba con el copy()
# Agregar columna
diana["shared"] = diana["room_type"] == "Shared room"
# Y, ahora sí, ordenar:
diana = diana.sort_values(by=["price", "shared", "review_scores_accuracy"], ascending=[True, False, False])
diana = diana.head(10)
diana

,id,host_id,neighbourhood_cleansed,property_type,room_type,accommodates,price,minimum_nights,maximum_nights,bedrooms,beds,cancellation_policy,number_of_reviews,review_scores_accuracy,shared
17593,39948133,126456380,Coyoacan,House,Private room,4,23,1,1125,1.0,2.0,moderate,5,10.0,False
19399,41909745,78891216,Benito Juarez,House,Private room,1,23,1,20,1.0,1.0,moderate,0,NaN,False
5713,20557042,64225887,Benito Juarez,Apartment,Private room,2,45,1,5,1.0,1.0,flexible,8,10.0,False
18657,41113140,320683349,Coyoacan,Loft,Entire home/apt,2,45,2,1125,0.0,2.0,moderate,7,10.0,False
19701,42193723,218080886,Benito Juarez,Guest suite,Entire home/apt,2,45,2,1125,0.0,1.0,flexible,2,10.0,False
5675,20476924,136277876,Cuauhtemoc,House,Private room,2,45,1,1125,1.0,1.0,flexible,0,NaN,False
19147,41642951,160469905,Benito Juarez,Apartment,Entire home/apt,4,46,3,1125,2.0,2.0,flexible,1,10.0,False
9428,27552465,163628660,Benito Juarez,Apartment,Private room,1,91,1,7,1.0,1.0,flexible,5,10.0,False
7728,23877762,179398369,Tlalpan,Loft,Shared room,1,159,1,1125,1.0,1.0,flexible,0,NaN,True
19674,42151081,257679722,Coyoacan,Apartment,Private room,2,159,1,1125,1.0,1.0,flexible,6,10.0,False


In [13]:
# Vamos a intentar hacerlo ahora sin agregar la columna extra al DataFrame
# (haciendo los cálculos implícitos)

# Filtrar
diana = airbnb[fp & fn]

In [14]:
# Construir un DataFrame auxiliar con las columnas de ordenación
df_p = diana["price"]
df_r = diana["room_type"].where(diana["room_type"]=="Shared room", other="ZZZ")
df_s = diana["review_scores_accuracy"]
orden = pd.concat([df_p, df_r, df_s], axis=1)
orden.columns = ["price", "room", "score"]
# Ordenar el DataFrame auxiliar
orden = orden.sort_values(by=["price", "room", "score"], ascending=[True, True, False])
# Usar los índices del DataFrame auxiliar para ordenar
diana = diana.loc[orden.index]
# Solo las primeras 10 opciones
diana = diana.head(10)
diana

,id,host_id,neighbourhood_cleansed,property_type,room_type,accommodates,price,minimum_nights,maximum_nights,bedrooms,beds,cancellation_policy,number_of_reviews,review_scores_accuracy
17593,39948133,126456380,Coyoacan,House,Private room,4,23,1,1125,1.0,2.0,moderate,5,10.0
19399,41909745,78891216,Benito Juarez,House,Private room,1,23,1,20,1.0,1.0,moderate,0,NaN
5713,20557042,64225887,Benito Juarez,Apartment,Private room,2,45,1,5,1.0,1.0,flexible,8,10.0
18657,41113140,320683349,Coyoacan,Loft,Entire home/apt,2,45,2,1125,0.0,2.0,moderate,7,10.0
19701,42193723,218080886,Benito Juarez,Guest suite,Entire home/apt,2,45,2,1125,0.0,1.0,flexible,2,10.0
5675,20476924,136277876,Cuauhtemoc,House,Private room,2,45,1,1125,1.0,1.0,flexible,0,NaN
19147,41642951,160469905,Benito Juarez,Apartment,Entire home/apt,4,46,3,1125,2.0,2.0,flexible,1,10.0
9428,27552465,163628660,Benito Juarez,Apartment,Private room,1,91,1,7,1.0,1.0,flexible,5,10.0
7728,23877762,179398369,Tlalpan,Loft,Shared room,1,159,1,1125,1.0,1.0,flexible,0,NaN
19674,42151081,257679722,Coyoacan,Apartment,Private room,2,159,1,1125,1.0,1.0,flexible,6,10.0


Sí se pudo ordenar sin agregar la columna auxiliar. Sin embargo, el proceso de crear el DataFrame auxiliar es mucho más complejo que crear la columna auxiliar. Si se hubiera hecho como se pretendía (haciendo los cálculos implícitos), el código hubiera quedado aún más complicado.